In [1]:
import os
import numpy as np
import json
import keras
from keras import backend as k
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator

data_path = '/home/rbodo/.snntoolbox/Datasets/imagenet'
train_path = os.path.join(data_path, 'training')
test_path = os.path.join(data_path, 'validation')
class_idx_path = os.path.join(data_path, 'imagenet_class_index_1000.json')
save_path = '/home/rbodo/.snntoolbox/data/imagenet/inceptionV3/keras_refined/'

Using Theano backend.


Using cuDNN version 5005 on context None
Mapped name None to device cuda: GeForce GTX 980 Ti (0000:01:00.0)


In [3]:
path_to_orig_model = os.path.join(save_path, '76.28_93.03.h5')
if os.path.isfile(path_to_orig_model):
    model = keras.models.load_model(path_to_orig_model)
else:
    # create the base pre-trained model
    model = InceptionV3()
    
    # Flip kernels (tensorflow uses correlation instead of convolution, like caffe)
    to_assign = []
    for layer in model.layers:
        if 'conv2d' in layer.name:
            original_kernel = k.get_value(layer.kernel)
            converted_kernel = original_kernel[::-1, ::-1, :, :]
            to_assign.append((layer.kernel, converted_kernel))
    k.batch_set_value(to_assign)
    
    model.save(path_to_orig_model)

GpuArrayException: b'out of memory'

In [22]:
target_size = (299, 299)
batch_size = 2

class_idx = json.load(open(class_idx_path, "r"))
classes = [class_idx[str(idx)][0] for idx in range(len(class_idx))]

datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

test_dataflow = datagen.flow_from_directory(
    test_path, target_size=target_size, classes=classes, batch_size=batch_size)

Found 50000 images belonging to 1000 classes.


In [23]:
train_dataflow = datagen.flow_from_directory(
    train_path, target_size=target_size, classes=classes, batch_size=batch_size)

Found 1281167 images belonging to 1000 classes.


In [8]:
from keras.constraints import unit_norm, max_norm
from keras.regularizers import l2

for layer in model.layers:
    # if 'conv' in layer.name:
    #     print("Modifying Conv layer")
    #     layer.kernel_constraint = unit_norm()
    #     layer.kernel_regularizer = l2(0.01)
    if 'batch' in layer.name:
        print("Modifying BatchNorm layer")
        # layer.beta_regularizer = l2(0.01)  # Should not be necessary
        layer.beta_constraint = max_norm(0.3)
    # if 'activ' in layer.name:
    #     # Try to see if we can make normalization unnecessary with this:
    #     print("Modifying Activation layer")
    #     layer.activity_regularizer = l2(0.01)

Modifying BatchNorm layer
Modifying BatchNorm layer
Modifying BatchNorm layer
Modifying BatchNorm layer
Modifying BatchNorm layer
Modifying BatchNorm layer
Modifying BatchNorm layer
Modifying BatchNorm layer
Modifying BatchNorm layer
Modifying BatchNorm layer
Modifying BatchNorm layer
Modifying BatchNorm layer
Modifying BatchNorm layer
Modifying BatchNorm layer
Modifying BatchNorm layer
Modifying BatchNorm layer
Modifying BatchNorm layer
Modifying BatchNorm layer
Modifying BatchNorm layer
Modifying BatchNorm layer
Modifying BatchNorm layer
Modifying BatchNorm layer
Modifying BatchNorm layer
Modifying BatchNorm layer
Modifying BatchNorm layer
Modifying BatchNorm layer
Modifying BatchNorm layer
Modifying BatchNorm layer
Modifying BatchNorm layer
Modifying BatchNorm layer
Modifying BatchNorm layer
Modifying BatchNorm layer
Modifying BatchNorm layer
Modifying BatchNorm layer
Modifying BatchNorm layer
Modifying BatchNorm layer
Modifying BatchNorm layer
Modifying BatchNorm layer
Modifying Ba

In [9]:
model.compile(SGD(lr=0.0001, momentum=0.9), 'categorical_crossentropy', metrics=['accuracy', keras.metrics.top_k_categorical_accuracy])

In [10]:
checkpoint = keras.callbacks.ModelCheckpoint('weights.{epoch:02d}-{val_acc:.2f}.h5', monitor='val_acc')

In [25]:
val_steps = 250
train_steps = 1281167 / batch_size
epochs = 10
history = model.fit_generator(train_dataflow, train_steps, epochs, verbose=0, callbacks=[checkpoint], validation_data=test_dataflow, validation_steps=val_steps)

/mnt/2646BAF446BAC3B9/VirtualEnvironments/anaconda3/envs/snntoolbox/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:692: UserWarning: Possibly corrupt EXIF data.  Expecting to read 2555904 bytes but only got 0. Skipping tag 0
  "Skipping tag %s" % (size, len(data), tag))


/mnt/2646BAF446BAC3B9/VirtualEnvironments/anaconda3/envs/snntoolbox/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:709: UserWarning: Corrupt EXIF data.  Expecting to read 4 bytes but only got 0. 
  warnings.warn(str(msg))


/mnt/2646BAF446BAC3B9/VirtualEnvironments/anaconda3/envs/snntoolbox/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:692: UserWarning: Possibly corrupt EXIF data.  Expecting to read 19660800 bytes but only got 0. Skipping tag 0
  "Skipping tag %s" % (size, len(data), tag))
/mnt/2646BAF446BAC3B9/VirtualEnvironments/anaconda3/envs/snntoolbox/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:692: UserWarning: Possibly corrupt EXIF data.  Expecting to read 18481152 bytes but only got 0. Skipping tag 0
  "Skipping tag %s" % (size, len(data), tag))
/mnt/2646BAF446BAC3B9/VirtualEnvironments/anaconda3/envs/snntoolbox/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:692: UserWarning: Possibly corrupt EXIF data.  Expecting to read 37093376 bytes but only got 0. Skipping tag 0
  "Skipping tag %s" % (size, len(data), tag))
/mnt/2646BAF446BAC3B9/VirtualEnvironments/anaconda3/envs/snntoolbox/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:692: UserWarning: Possibly corrupt EXIF data.  Expect

/mnt/2646BAF446BAC3B9/VirtualEnvironments/anaconda3/envs/snntoolbox/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:692: UserWarning: Possibly corrupt EXIF data.  Expecting to read 1835008 bytes but only got 0. Skipping tag 0
  "Skipping tag %s" % (size, len(data), tag))


In [ ]:
score = model.evaluate_generator(test_dataflow, 25)

In [42]:
score

[1.0380793774127961, 0.76200000286102298, 0.94399999618530273]

In [ ]:
model.save(os.path.join(save_path, 'inception_refined.h5'))